In [ ]:
!pip install twikit

# **Import Library dan token twitter `Twikit` mengharuskan user menginputkan token sehingga tidak bisa anonim**

In [ ]:
import csv
import nest_asyncio
import asyncio
import random
import math
from twikit import Client, TooManyRequests

nest_asyncio.apply()
# SETUP CLIENT + COOKIE
client = Client("en-US")
client.set_cookies({
    "auth_token": "d125d285011b59146b60cdcd6edd9c82dca055c9",
    "ct0": "59e9745d43723d4cb98f25be06a03b19dc235dbb0f0410ca8b0b49f8231a184b2bc947fcc2fd77668b90979426c200556e89b9d9dbd0089e9676e867fd25a2fe82a0380ac635254a31df26275aa7854e",
    "twid": "u%3D1645176567038091265",
    "gt": "1989338289653047338",
    "guest_id": "v1%3A175121650454954896",
    "guest_id_ads": "v1%3A175121650454954896",
    "guest_id_marketing": "v1%3A175121650454954896",
    "kdt": "HZdPjfR2pqsvUhxsO5GasalB5x7wYSzJ8sBbSx85",
    "lang": "en",
    "personalization_id": "v1_JcinIxPjnVOOctgz7rKeiQ==",
    "cf_bm": "sjuQ5SgqIGcWGuc4AGCyaC42n3AikDN522y1tic1cGI-1763135431.7418268-1.0.1.1-3bCnpzTDAGCtwt0vvIoOgF_Li.Jdb5N0EUXsxh9yF6D9Kr7o3ZWHdWOGMmMYdqgGcF5rTpDcqAfpWxucyTaZlbd98K41EE4y6jyCmrn4tZwRPkpnfZsJPbUDKIDjxIY5",
    "cuid": "f6d3d51a5a0345ee9e273cb98b2c5068",
    "att": "1-PToudQBU2NDJeJZeJ8k2vit0taZstJ225z7kN54g",
    "external_referer": "padhuUp37zhD6%2F29CpQtyhGQCUl05AFo|0|8e8t2xd8A2w%3D",
    "g_state": "{\"i_l\":0,\"i_ll\":1763130464264}",
    "gt": "1989338289653047338",
})


# **Buat parameter untuk scrapping**
## **saya memberi delay lebih lama agar request tetap berjalan dan tidak di anggap spam**

In [ ]:
# Config
BATCH_SIZE = 20
TARGET_PER_KEYWORD = 500
CHECKPOINT_EVERY = 50
MIN_DELAY = 10
MAX_DELAY = 20
MAX_RETRIES = 6
CSV_FILENAME = "tweet_gojek_grab.csv"

def write_header_if_needed(filename):
    try:
        with open(filename, "x", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["keyword", "tweet"])
    except FileExistsError:
        pass

def append_rows(filename, rows):
    with open(filename, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

In [ ]:
# scraping dengan pagination, backoff, dan checkpointing
async def scrape_keyword(keyword, target=TARGET_PER_KEYWORD):
    print(f"Start scraping keyword: {keyword} (target {target})")
    results = []
    cursor = None
    consecutive_empty = 0
    write_header_if_needed(CSV_FILENAME)

    while len(results) < target:
        attempt = 0
        while True:
            try:
                response = await client.search_tweet(
                    f"{keyword} lang:id",
                    product="Latest",
                    count=BATCH_SIZE,
                    cursor=cursor
                )
                tweets = response if response is not None else []
                break

            except TooManyRequests:
                attempt += 1
                backoff = min(60, 2 ** attempt + random.random() * 5)
                print(f"429 Rate limit. Backoff for {backoff:.1f}s (attempt {attempt})")
                await asyncio.sleep(backoff)
                if attempt >= MAX_RETRIES:
                    print("Max retries reached for 429. Exiting this keyword early.")
                    return results

            except Exception as e:
                attempt += 1
                backoff = min(60, (2 ** attempt) + random.random() * 5)
                print(f"transient error: {e!r}. Backoff {backoff:.1f}s (attempt {attempt})")
                await asyncio.sleep(backoff)
                if attempt >= MAX_RETRIES:
                    print("Max retries reached for errors. Exiting this keyword early.")
                    return results

        if not tweets:
            consecutive_empty += 1
            print(f"ℹNo tweets returned in this batch (empty count={consecutive_empty}).")
            if consecutive_empty >= 4:
                print("ℹToo many consecutive empty batches — stopping pagination for this keyword.")
                break
            await asyncio.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            continue
        else:
            consecutive_empty = 0

        batch_texts = []
        for t in tweets:
            try:
                text = t.text
            except Exception:
                text = str(t)
            batch_texts.append(text)

        rows = []
        for txt in batch_texts:
            results.append(txt)
            rows.append([keyword.replace(" lang:id",""), txt])

        if rows:
            append_rows(CSV_FILENAME, rows)


        print(f"  got {len(batch_texts)} tweets (total collected: {len(results)}/{target})")

        try:
            next_cursor = getattr(tweets, "next_cursor", None)
            if next_cursor is None and isinstance(response, dict):
                next_cursor = response.get("next_cursor") or response.get("cursor")
        except Exception:
            next_cursor = None

        if not next_cursor:
            if len(results) >= target:
                break
            cursor = None
            print("no next_cursor; will try a few more small batches before stopping.")
        else:
            cursor = next_cursor
        await asyncio.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
        if len(results) >= target:
            break

    print(f"Finished {keyword}: collected {len(results)} tweets.")
    return results[:target]

In [ ]:
# main driver
async def main():
    write_header_if_needed(CSV_FILENAME)
    keywords = ["gojek", "grab"]

    final = {}
    for kw in keywords:
        tweets = await scrape_keyword(kw, TARGET_PER_KEYWORD)
        final[kw] = tweets
        cooldown = random.uniform(5, 12)
        print(f"Cooldown {cooldown:.1f}s before next keyword.")
        await asyncio.sleep(cooldown)

    print("All done. Summary:")
    for k, v in final.items():
        print(f" - {k}: {len(v)} tweets collected")
await main()

Start scraping keyword: gojek (target 500)
  got 19 tweets (total collected: 19/500)
  got 20 tweets (total collected: 39/500)
  got 20 tweets (total collected: 59/500)
  got 20 tweets (total collected: 79/500)
  got 20 tweets (total collected: 99/500)
  got 20 tweets (total collected: 119/500)
  got 20 tweets (total collected: 139/500)
  got 20 tweets (total collected: 159/500)
  got 20 tweets (total collected: 179/500)
  got 20 tweets (total collected: 199/500)
  got 20 tweets (total collected: 219/500)
  got 20 tweets (total collected: 239/500)
  got 20 tweets (total collected: 259/500)
  got 20 tweets (total collected: 279/500)
  got 20 tweets (total collected: 299/500)
  got 20 tweets (total collected: 319/500)
  got 20 tweets (total collected: 339/500)
  got 20 tweets (total collected: 359/500)
  got 20 tweets (total collected: 379/500)
  got 19 tweets (total collected: 398/500)
  got 20 tweets (total collected: 418/500)
  got 20 tweets (total collected: 438/500)
  got 20 tweets 